<a href="https://colab.research.google.com/github/Shi-pra-19/dl-genai-project/blob/main/roberta_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ROBERTA TRAINING

##Overview

After training a single RoBERTa-Large baseline model for 10 epochs,resulting in 0.865 score on leaderboard, we extend the setup by training three separate models with different random seeds (42, 2025, 123) to improve stability and generalization. The training uses a long sequence length (MAX_LEN = 512), gradient accumulation for an effective larger batch size, and a 10-epoch schedule with a cosine LR scheduler. To better handle class imbalance, we replace the default BCE loss with Focal Loss (α = 0.25, γ = 2.0) through a custom Trainer. Each model is trained using the same hyperparameters and tokenization settings, and the best results per epoch are logged via Weights & Biases. After training, we compute optimal per-label thresholds using validation predictions, and then create an ensemble by averaging model probabilities across all seeds. Finally, the ensemble is used to generate test predictions, apply tuned thresholds, and create the final submission CSV along with saved thresholds for reproducibility.

## IMPORTS

In [ ]:
!pip install wandb transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 40.8 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 19.0.1
    Uninstalling pyarrow-19.0.1:
      Successfully uninstalled pyarrow-19.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
pylibcudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
cudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
cudf-polars-cu12 25.6.0 requires pylibcudf-cu12==25.6.*, but you have pylibcudf-cu12 25.2.2 which is incompatible.


In [ ]:
import pandas as pd
import numpy as np
import torch
import random
from torch import nn
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    Trainer,
    TrainingArguments,
)
from datasets import Dataset, DatasetDict
import wandb
import os

2025-11-28 16:14:57.390889: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764346497.589425      20 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764346497.648491      20 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

## VARIABLES

In [ ]:
wandb.login(key='16a767377715590d2d5fe6351174577f96db6dc6')

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


True

In [ ]:
df = pd.read_csv('/kaggle/input/dlgenai/augmented_train.csv')

In [ ]:
MAX_LEN = 512
BATCH_SIZE = 8
GRADIENT_ACCUMULATION_STEPS = 4
EPOCHS = 10
LEARNING_RATE = 2e-5
DEBERTA_MODEL = "roberta-large"
NUM_LABELS = 5
LABEL_COLS = ["anger", "joy", "fear", "surprise", "sadness"]
SEEDS = [42, 2025, 123]
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

#CUSTOM LOSS

**Custom Focal Loss module that down-weights easy examples and focuses training on harder, misclassified samples using α = 0.25 and γ = 2.0. It then integrates this loss into a CustomTrainer by overriding compute_loss, replacing the default BCE loss with Focal Loss for more robust multi-label emotion classification.**

**Focal Loss is used to handle class imbalance by reducing the contribution of easy negatives and pushing the model to learn harder, minority-class examples more effectively. Integrating it into the CustomTrainer helps improve recall and overall F1 scores for under-represented emotion labels.**

In [ ]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, inputs, targets):
        BCE_loss = nn.BCEWithLogitsLoss(reduction='none')(inputs, targets)
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss
        return F_loss.mean()

In [ ]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fct = FocalLoss(alpha=0.25, gamma=2.0)
        loss = loss_fct(logits, labels.float())
        return (loss, outputs) if return_outputs else loss

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = (torch.sigmoid(torch.tensor(logits)) >= 0.5).int().numpy()
    labels = labels.astype(int)
    macro_f1 = f1_score(labels, preds, average="macro", zero_division=0)
    per_label_f1 = f1_score(labels, preds, average=None, zero_division=0)
    result = {"macro_f1": macro_f1}
    for lbl, score in zip(LABEL_COLS, per_label_f1):
        result[f"f1_{lbl}"] = score
    wandb.log(result)
    return result

##Optimal Thresholds

**This function searches for the best decision threshold for each emotion label by testing values from 0.20 to 0.75 and selecting the one that produces the highest F1 score on validation data. Instead of using a fixed 0.5 cutoff, it computes label-specific thresholds to better match each class's distribution. This improves multi-label performance because different emotions have different score ranges, and optimized thresholds significantly boost overall F1 and recall.**

In [ ]:
def find_optimal_thresholds(val_probs, val_labels):
    thresholds = {}
    print("Finding Optimal Thresholds per Label")
    print("="*50)

    for i, label in enumerate(LABEL_COLS):
        best_f1 = 0
        best_thresh = 0.5

        for thresh in np.arange(0.2, 0.8, 0.05):
            preds = (val_probs[:, i] >= thresh).astype(int)
            f1 = f1_score(val_labels[:, i], preds, zero_division=0)

            if f1 > best_f1:
                best_f1 = f1
                best_thresh = thresh

        thresholds[label] = best_thresh
        print(f"{label:10s}: threshold={best_thresh:.2f}, F1={best_f1:.4f}")

    print("="*50 + "\n")
    return thresholds

## DATA PREPARATION FOR TRAINING

In [ ]:
texts = df["text"].astype(str).tolist()
labels = df[LABEL_COLS].values

df = pd.DataFrame({"text": texts})
for i, col in enumerate(LABEL_COLS):
    df[col] = labels[:, i]

In [ ]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(test_df, test_size=0.5, random_state=42)

print(f"Train: {len(train_df)} | Val: {len(val_df)} | Test: {len(test_df)}")

Train: 12501 | Val: 1563 | Test: 1563


**Converts the data into HuggingFace Dataset format, applies tokenization, and attaches the emotion label vectors to each sample. Enables dynamic padding for efficient batching and smoother model training.**

In [ ]:
dataset = DatasetDict({
    "train": Dataset.from_pandas(train_df),
    "validation": Dataset.from_pandas(val_df),
    "test": Dataset.from_pandas(test_df),
})

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(DEBERTA_MODEL)

tokenized_datasets = dataset.map(
    lambda examples: tokenizer(examples["text"], truncation=True),
    batched=True,
)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/12501 [00:00<?, ? examples/s]

Map:   0%|          | 0/1563 [00:00<?, ? examples/s]

Map:   0%|          | 0/1563 [00:00<?, ? examples/s]

In [ ]:
def set_labels(example):
    example["labels"] = [float(example[col]) for col in LABEL_COLS]
    return example

tokenized_datasets = tokenized_datasets.map(set_labels)

# dynamic padding per batch
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/12501 [00:00<?, ? examples/s]

Map:   0%|          | 0/1563 [00:00<?, ? examples/s]

Map:   0%|          | 0/1563 [00:00<?, ? examples/s]

##TRAINING

1. Trains the RoBERTa model multiple times using different random seeds to improve robustness and reduce variance.

2. Logs each run to Weights & Biases with its own config for tracking experiments.

3. Initializes a fresh model and training setup for every seed to ensure independent runs.

4. Trains with Focal Loss, cosine LR scheduling, gradient accumulation, and logs metrics.

5. Evaluates each model on the test set and stores the seed-wise results.

6. Generates validation predictions
for threshold tuning and test predictions for ensembling.

7. Saves each trained model and
tokenizer, uploads them to W&B as artifacts for reproducibility.

In [ ]:
all_val_predictions = []
all_test_predictions = []
seed_results = []

In [ ]:
for seed_idx, seed in enumerate(SEEDS):
    print(f"TRAINING MODEL WITH SEED {seed} ({seed_idx+1}/{len(SEEDS)})")

    set_seed(seed)

    # Initialize wandb
    wandb.init(
        project="23f3001910-t32025",
        name=f"deberta-large-seed{seed}",
        config={
            "seed": seed,
            "batch_size": BATCH_SIZE,
            "gradient_accumulation_steps": GRADIENT_ACCUMULATION_STEPS,
            "effective_batch_size": BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS,
            "epochs": EPOCHS,
            "learning_rate": LEARNING_RATE,
            "focal_loss": True,
            "lr_scheduler": "cosine",
            "warmup_ratio": 0.1,
        },
    )


    model = AutoModelForSequenceClassification.from_pretrained(
        DEBERTA_MODEL,
        problem_type="multi_label_classification",
        num_labels=NUM_LABELS,
    )

#training related arguments
    training_args = TrainingArguments(
        output_dir=f"./results_seed_{seed}",
        eval_strategy="epoch",
        save_strategy="no",
        save_total_limit=1,
        learning_rate=LEARNING_RATE,
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=BATCH_SIZE,
        gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
        num_train_epochs=EPOCHS,
        weight_decay=0.01,
        load_best_model_at_end=False,
        metric_for_best_model="macro_f1",
        greater_is_better=True,
        logging_dir=f"./logs_seed_{seed}",
        logging_steps=50,
        report_to=["wandb"],
        warmup_ratio=0.1,
        lr_scheduler_type="cosine",
        seed=seed,
    )

#using custom trainer

    trainer = CustomTrainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["validation"],
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )


    trainer.train()

    metrics = trainer.evaluate(tokenized_datasets["test"])
    print(f"\nSeed {seed} Test Metrics: {metrics}")
    seed_results.append(metrics)

    # Get validation predictions for threshold tuning
    val_outputs = trainer.predict(tokenized_datasets["validation"])
    val_probs = torch.sigmoid(torch.tensor(val_outputs.predictions)).numpy()
    all_val_predictions.append(val_probs)

    # Get test predictions
    test_outputs = trainer.predict(tokenized_datasets["test"])
    test_probs = torch.sigmoid(torch.tensor(test_outputs.predictions)).numpy()
    all_test_predictions.append(test_probs)

    # Save model
    model_dir = f"./deberta_large_seed_{seed}"
    trainer.save_model(model_dir)
    tokenizer.save_pretrained(model_dir)

    artifact = wandb.Artifact(f"deberta_large_seed_{seed}", type="model")
    artifact.add_dir(model_dir)
    wandb.log_artifact(artifact)

    wandb.finish()

TRAINING MODEL WITH SEED 42 (1/3)


wandb: Tracking run with wandb version 0.21.0
wandb: Run data is saved locally in /kaggle/working/wandb/run-20251128_161528-ppmns36j
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run deberta-large-seed42
wandb: ⭐️ View project at https://wandb.ai/23f3001910-indian-institute-of-technology-madras/23f3001910-t32025
wandb: 🚀 View run at https://wandb.ai/23f3001910-indian-institute-of-technology-madras/23f3001910-t32025/runs/ppmns36j


model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_20/1058363764.py:58: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead.
  trainer = CustomTrainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss,Macro F1,F1 Anger,F1 Joy,F1 Fear,F1 Surprise,F1 Sadness
1,0.032000,0.031522,0.651856,0.535050,0.645995,0.798676,0.704412,0.575148
2,0.028200,0.026791,0.692067,0.568940,0.765160,0.795181,0.631356,0.699697
3,0.019400,0.026307,0.776643,0.751740,0.764423,0.835314,0.774697,0.757042
4,0.013700,0.028170,0.799339,0.771196,0.812568,0.855263,0.777577,0.780090
5,0.008900,0.030135,0.811196,0.797701,0.781903,0.848347,0.818419,0.809612
6,0.004900,0.036846,0.817447,0.798603,0.832787,0.863946,0.807240,0.784661
7,0.003200,0.040210,0.825842,0.827283,0.813747,0.864436,0.817690,0.806055
8,0.001800,0.042807,0.831763,0.815068,0.826840,0.871570,0.834215,0.811122
9,0.001400,0.043442,0.831712,0.819304,0.827887,0.868406,0.829052,0.813910
10,0.001100,0.043566,0.832268,0.822870,0.829322,0.867001,0.827891,0.814259


...

TRAINING MODEL WITH SEED 123 (3/3)


wandb: Tracking run with wandb version 0.21.0
wandb: Run data is saved locally in /kaggle/working/wandb/run-20251128_173643-204sk3jf
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run deberta-large-seed123
wandb: ⭐️ View project at https://wandb.ai/23f3001910-indian-institute-of-technology-madras/23f3001910-t32025
wandb: 🚀 View run at https://wandb.ai/23f3001910-indian-institute-of-technology-madras/23f3001910-t32025/runs/204sk3jf
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_20/1058363764.py:58: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead.
  trainer =

Epoch,Training Loss,Validation Loss,Macro F1,F1 Anger,F1 Joy,F1 Fear,F1 Surprise,F1 Sadness
1,0.032500,0.028880,0.674142,0.614583,0.665813,0.786885,0.668571,0.634855
2,0.026300,0.027159,0.721185,0.720093,0.763968,0.797583,0.608305,0.715976
3,0.020200,0.025853,0.770120,0.727273,0.781633,0.835075,0.747075,0.759542
4,0.012200,0.027191,0.803334,0.786036,0.805017,0.844866,0.788530,0.792219
5,0.007700,0.032422,0.810586,0.795580,0.805281,0.849252,0.791859,0.810959
6,0.004500,0.037646,0.821354,0.822346,0.827957,0.853075,0.809050,0.794340
7,0.002400,0.044600,0.821903,0.809195,0.828092,0.860571,0.809353,0.802303
8,0.001300,0.047364,0.828472,0.829978,0.831169,0.866967,0.805505,0.808743
9,0.000900,0.048332,0.831668,0.825893,0.833876,0.872281,0.814480,0.811808
10,0.000900,0.048481,0.830986,0.825503,0.830435,0.871309,0.816621,0.811060



Seed 123 Test Metrics: {'eval_loss': 0.048606086522340775, 'eval_macro_f1': 0.825013808117192, 'eval_f1_anger': 0.8211009174311927, 'eval_f1_joy': 0.8145833333333333, 'eval_f1_fear': 0.8559670781893003, 'eval_f1_surprise': 0.8106565176022835, 'eval_f1_sadness': 0.8227611940298507, 'eval_runtime': 8.045, 'eval_samples_per_second': 194.283, 'eval_steps_per_second': 24.363, 'epoch': 10.0}


wandb: Adding directory to artifact (./deberta_large_seed_123)... Done. 4.2s
wandb: uploading artifact deberta_large_seed_123; uploading wandb-summary.json; uploading config.yaml
wandb: uploading artifact deberta_large_seed_123
wandb: uploading data
wandb:                                                                                
wandb: 
wandb: Run history:
wandb:           eval/f1_anger ▁▄▅▇▇█▇████
wandb:            eval/f1_fear ▁▂▅▆▆▆▇███▇
wandb:             eval/f1_joy ▁▅▆▇▇█████▇
wandb:         eval/f1_sadness ▁▄▆▇█▇▇▇███
wandb:        eval/f1_surprise ▃▁▆▇▇██████
wandb:               eval/loss ▂▁▁▁▃▅▇████
wandb:           eval/macro_f1 ▁▃▅▇▇██████
wandb:            eval/runtime ▇█████████▁
wandb: eval/samples_per_second ▁▁▁▁▁▁▁▁▁▁█
wandb:   eval/steps_per_second ▁▁▁▁▁▁▁▁▁▁█
wandb:                f1_anger ▁▄▅▇▇█▇██████
wandb:                 f1_fear ▁▂▅▆▆▆▇███▇█▇
wandb:                  f1_joy ▁▅▆▇▇█████▇█▇
wandb:              f1_sadness ▁▄▆▇█▇▇▇█████
wandb:             f1_sur

In [ ]:
ensemble_val_probs = np.mean(all_val_predictions, axis=0)
val_labels = tokenized_datasets["validation"][LABEL_COLS[0]]
val_labels_array = np.array([tokenized_datasets["validation"][col] for col in LABEL_COLS]).T

# Find optimal thresholds using ensemble validation predictions
optimal_thresholds = find_optimal_thresholds(ensemble_val_probs, val_labels_array)

# Average test predictions from all seeds
ensemble_test_probs = np.mean(all_test_predictions, axis=0)

# Apply optimal thresholds
ensemble_preds = np.zeros_like(ensemble_test_probs)
for i, label in enumerate(LABEL_COLS):
    ensemble_preds[:, i] = (ensemble_test_probs[:, i] >= optimal_thresholds[label]).astype(int)

# Calculate ensemble metrics
test_labels = tokenized_datasets["test"][LABEL_COLS[0]]
test_labels_array = np.array([tokenized_datasets["test"][col] for col in LABEL_COLS]).T

ensemble_macro_f1 = f1_score(test_labels_array, ensemble_preds, average="macro", zero_division=0)
ensemble_per_label_f1 = f1_score(test_labels_array, ensemble_preds, average=None, zero_division=0)

Finding Optimal Thresholds per Label
anger     : threshold=0.40, F1=0.8389
joy       : threshold=0.40, F1=0.8414
fear      : threshold=0.45, F1=0.8800
surprise  : threshold=0.40, F1=0.8451
sadness   : threshold=0.45, F1=0.8226



In [ ]:
print(f"Macro F1: {ensemble_macro_f1:.4f}")
for label, f1 in zip(LABEL_COLS, ensemble_per_label_f1):
    print(f"{label:10s}: {f1:.4f}")

for seed, result in zip(SEEDS, seed_results):
    print(f"Seed {seed}: Macro F1 = {result['eval_macro_f1']:.4f}")

Macro F1: 0.8375
anger     : 0.8261
joy       : 0.8260
fear      : 0.8662
surprise  : 0.8438
sadness   : 0.8254
Seed 42: Macro F1 = 0.8244
Seed 2025: Macro F1 = 0.8170
Seed 123: Macro F1 = 0.8250


## RESULTS

**SEED 42**
- Macro F1: 0.8244

- Anger F1: 0.8150

- Joy F1: 0.8029

- Fear F1: 0.8578

- Surprise F1: 0.8321

- Sadness F1: 0.8143

- Test Loss: 0.0446

Train, Test and Validation Data performed similarly with a score around ~ 0.825

**SEED 2025**
- Macro F1: 0.8170

- Anger F1: 0.7991

- Joy F1: 0.8

- Fear F1: 0.85

- Surprise F1: 0.83

- Sadness F1: 0.8019

- Test Loss: 0.0468

Train, Test and Validation Data performed similarly with a score around ~ 0.81

**SEED 123**
- Macro F1: 0.82

- Anger F1: 0.82

- Joy F1: 0.81

- Fear F1: 0.85

- Surprise F1: 0.82

- Sadness F1: 0.81

- Test Loss: 0.0461

Train, Test and Validation Data performed similarly with a score around ~ 0.82

**Optimal Thresholds**
- Macro F1: 0.8375
- anger     : 0.8261
- joy       : 0.8260
- fear      : 0.8662
- surprise  : 0.8438
- sadness   : 0.8254
- Seed 42: Macro F1 = 0.8244
- Seed 2025: Macro F1 = 0.8170
- Seed 123: Macro F1 = 0.8250

## INFERENCE

In [ ]:
TEST_PATH = "/kaggle/input/dlgenai/test_clean.csv"
test = pd.read_csv(TEST_PATH)
texts = test["text"].astype(str).tolist()

In [ ]:
all_model_preds = []

for seed in SEEDS:
    print(f"Loading model with seed {seed}...")
    model_dir = f"./deberta_large_seed_{seed}"
    model = AutoModelForSequenceClassification.from_pretrained(model_dir)
    model.to(DEVICE)
    model.eval()

    batch_preds = []
    INFERENCE_BATCH_SIZE = 16

    for i in range(0, len(texts), INFERENCE_BATCH_SIZE):
        batch_texts = texts[i:i+INFERENCE_BATCH_SIZE]
        inputs = tokenizer(batch_texts, return_tensors="pt", truncation=True, padding=True).to(DEVICE)

        with torch.no_grad():
            outputs = model(**inputs)
            probs = torch.sigmoid(outputs.logits).cpu().numpy()
            batch_preds.extend(probs)

    all_model_preds.append(np.array(batch_preds))
    print(f"Predictions generated")

Loading model with seed 42...
Predictions generated
Loading model with seed 2025...
Predictions generated
Loading model with seed 123...
Predictions generated


In [ ]:
final_probs = np.mean(all_model_preds, axis=0)

# Apply optimal thresholds
final_preds = np.zeros_like(final_probs)
for i, label in enumerate(LABEL_COLS):
    final_preds[:, i] = (final_probs[:, i] >= optimal_thresholds[label]).astype(int)

# Create submission
for i, label in enumerate(LABEL_COLS):
    test[label] = final_preds[:, i].astype(int)

columns_to_keep = ['id', 'anger', 'fear', 'joy', 'sadness', 'surprise']
test = test[columns_to_keep]

In [ ]:
test.to_csv("submission_seed_ensemble.csv", index=False)